In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from vector_search import VectorSearch
from typing import Optional, List, Tuple, Dict, Any
from datetime import datetime, timedelta
from core import user_msg,assistant_msg,system_msg
from qdrant_client import models

In [3]:
from vexa import VexaAPI
vexa = VexaAPI()
await vexa.get_user_info()

user_id = vexa.user_id
user_name = vexa.user_name
import asyncio
from vector_search import VectorSearch,build_context_string
from typing import Optional, Union, List
import pandas as pd

User information retrieved successfully.


In [4]:
vector_search = VectorSearch(gpu_device='0')

In [12]:
async def search_documents(vector_search_query: Optional[str] = None,
                           vector_search: VectorSearch = vector_search,
                           user_id: Optional[str] = user_id, 
                           user_name: Optional[str] = user_name,
                           k: int = 100,
                           include_summary: bool = False,
                           start: Optional[datetime] = None,
                           end: Optional[datetime] = None,
                           speakers_include: Optional[List[str]] = None,
                           speakers_exclude: Optional[List[str]] = None,
                           ) -> List[Tuple[Dict[str, Any], float, str]]:

    filter_conditions = []
    if speakers_include:
        filter_conditions.append(models.FieldCondition(
            key="speakers", 
            match=models.MatchAny(any=speakers_include)
        ))
    
    if speakers_exclude:
        filter_conditions.append(models.FieldCondition(
            key="speakers", 
            match=models.MatchNone(none=speakers_exclude)
        ))
    if not include_summary:
        filter_conditions.append(models.FieldCondition(key="type", match=models.MatchValue(value="point")))
    
    if start:
        adjusted_start = start - timedelta(days=1) if isinstance(start, datetime) else datetime.fromisoformat(start) - timedelta(days=1)
        filter_conditions.append(models.FieldCondition(
            key="start_datetime",
            range=models.Range(gte=int(adjusted_start.timestamp()))
        ))
    if end:
        adjusted_end = end + timedelta(days=1) if isinstance(end, datetime) else datetime.fromisoformat(end) + timedelta(days=1)
        filter_conditions.append(models.FieldCondition(
            key="start_datetime",
            range=models.Range(lte=int(adjusted_end.timestamp()))
        ))
    
    if user_id:
        if isinstance(user_id, str):
            filter_conditions.append(models.FieldCondition(key="user_id", match=models.MatchValue(value=user_id)))
        else:
            filter_conditions.append(models.FieldCondition(key="user_id", match=models.MatchAny(any=user_id)))
    
    if user_name:
        if isinstance(user_name, str):
            filter_conditions.append(models.FieldCondition(key="user_name", match=models.MatchValue(value=user_name)))
        else:
            filter_conditions.append(models.FieldCondition(key="user_name", match=models.MatchAny(any=user_name)))


    if vector_search_query:
        query_vector = await vector_search.get_embeddings([vector_search_query])
        search_result = await vector_search.qdrant_client.search(
            collection_name=vector_search.collection_name,
            query_vector=query_vector[0].tolist(),
            query_filter=models.Filter(must=filter_conditions),
            limit=k
        )
    else:
        search_result = await vector_search.qdrant_client.scroll(
            collection_name=vector_search.collection_name,
            scroll_filter=models.Filter(must=filter_conditions),
            limit=k,
            with_payload=True,
            with_vectors=True
        )
        search_result = search_result[0]  # Unpack the tuple returned by scroll

    return [(hit.payload, getattr(hit, 'score', 1.0), hit.id) for hit in search_result]

In [65]:
def build_context_string(summaries, points_by_meeting=None, only_summaries=False, include_all_summaries=True):
    context = []
    meeting_counter = 1
    meeting_ids = []
    for summary in summaries:
        meeting_id = summary['meeting_session_id']
        if include_all_summaries or (points_by_meeting and meeting_id in points_by_meeting):
            start_datetime = pd.to_datetime(datetime.fromtimestamp(summary['start_datetime']))
            context.append(f"# [{meeting_counter}] {summary['meeting_name']} ({start_datetime.strftime('%Y-%m-%d %H:%M')}) participants: {summary['speakers']}\n{summary['content']}")
            
            if not only_summaries and points_by_meeting and meeting_id in points_by_meeting:
                context.append("## Related Quotes:")
                for point in points_by_meeting[meeting_id]:
                    context.append(f"{point['qoutes']}")
            
            context.append("\n---\n")
            meeting_ids.append(meeting_id)
            meeting_counter += 1

    # Join context
    full_context = "\n".join(context)
    return full_context, meeting_ids

In [14]:
summaries = await vector_search.get_summaries(user_id=user_id)

In [15]:
speakers = await vector_search.get_speakers_by_user_id(user_id)

In [16]:
sorted(speakers)

['AI Marketing Directory',
 'Ahmed Abdelaziz',
 'Alex Loktev',
 'Alex Shevliakov',
 'Anastasiia GULIAEVA',
 'Andrey Pisankin',
 'Andy Black',
 'Angelina Geru',
 'Daniil Andreev',
 'David Sterry',
 'Dmitrii Bashkirov',
 'Dmitrii Chistov',
 'Dmitriy Grankin',
 'Dmitriy Grankin (Presentation)',
 'Dmitry Grankin',
 'Dmitry Grankin (Presentation)',
 'Dmitry Grankin (Your presentation)',
 'Dmtiry Grankin',
 'Ethan Noah',
 'Eugene Tartakovsky',
 'Igor Bessonov',
 'Igor Vinidiktov',
 'Ilia Semukhin',
 'Ilia Semukhin (Presentation)',
 'Ilya Semukhin',
 'Jenish Mursidinov',
 'Julia Zakharova',
 'Karine Stepanyan',
 'Lara Vargas',
 'Lidiia Abramova',
 'Maksim Jmihov',
 'Matt Lewis',
 'Max Mironov',
 'Mayank Tayal',
 'Murat Sultanov',
 'Nadya Gorodetskaya',
 'Nick Frolov',
 'Nikolay',
 'Oleg Maleev',
 'Olga Miller',
 'Olga Mykhoparkina',
 'Olga Nemirovskaya',
 'Rick Tousseyn',
 'Rick Tousseyn (Presentation)',
 'Robert Hangu',
 'Sergey Fofanov',
 'Sergey Ryabenko',
 'Sergio Goriachev',
 'Shota Arab

In [58]:
coworkers = ['Lara Vargas','Karine Stepanyan','Sergey Fofanov','Sergey Fofanov','Olga Nemirovskaya','Sergey Ryabenko','Andrey Pisankin']

In [66]:
# exclude coworkers
speakers = [speaker for speaker in speakers if speaker not in coworkers]

In [67]:
q = ''

In [68]:
# include coworkers
speakers = [speaker for speaker in speakers if speaker  in coworkers]

In [69]:
speakers = [speaker for speaker in speakers if speaker not in coworkers]
points_with_scores = await search_documents('seo tools',speakers_include=speakers)
points_with_scores
points_by_meeting = {}
for point, score, id in points_by_meeting:
    if score > 0.1:
        meeting_id = point['meeting_session_id']
        if meeting_id not in points_by_meeting:
            points_by_meeting[meeting_id] = []
        points_by_meeting[meeting_id].append(point)

full_context, meeting_ids = build_context_string(summaries, points_by_meeting, only_summaries=False, include_all_summaries=False)
df = pd.DataFrame([
    {
        'qoutes': point['qoutes'],
        'score': score,
        'id': id,
        'meeting_session_id': point['meeting_session_id'],
        'start_datetime': point['start_datetime'],
        'speakers': ', '.join(point['speakers']),
        'user_id': point['user_id'],
        'user_name': point['user_name']
    }
    for point, score, id in points_with_scores
])

# Convert start_datetime to a readable format
df['start_datetime'] = pd.to_datetime(df['start_datetime'], unit='s')

# Sort the DataFrame by score in descending order
df = df.sort_values('score', ascending=False)
grouped = df.sort_values('start_datetime').groupby('meeting_session_id')

# Aggregate the data
result = grouped.agg({
    'qoutes': lambda x: ' '.join(x),
    'score': 'max',
    'start_datetime': 'first',
    'speakers': 'first',
}).reset_index(drop=True)

# Sort the result by total score in descending order
result = result.sort_values('score', ascending=False)

In [70]:
pd.set_option('display.max_colwidth', None)

In [71]:
result

qoutes  \
6   Dmitriy Grankin:  So you are in, uh, so your, uh, your full-time job is this, uh, uh, team Lewis. Dmitriy Grankin:  Yeah. Dmitriy Grankin:  So team Lewis is like a global agency. Rick Tousseyn:  Um, I'm under a head of marketing, so I run the SEO division. Rick Tousseyn:  So, um, unfortunately we already did. Rick Tousseyn:  um the marketing budget of the year so i'm not allowed to spend more than the limit so semrush is as part of the tool tools that i use i also use surf for seo the one that i showed you but every year our fiscal year starts in end of august so september is the year that i need to present like the budget for the next year oh yes so for next year it's over already for yeah so for this year it's over but we have to wait i have to wait another year to present to to my head of marketing like the budget of the year so that's kind of like the problem i can't purchase any new tools okay um but i can give like recommendations but yeah Dmitriy Grankin:  will be yeah for next year so very I was not even looking for that but but really interesting and all Rick Tousseyn:  all seo companies work like this so they have like a fiscal year some companies they start the year in january the the financial year but some companies they started in a different month and we start our fiscal year in September. Rick Tousseyn:  So September is the new year for the budget. Rick Tousseyn:  And if you know this, you can actually actually just ask any SEO expert, Hey, when, when is the time that you, you are looking to do the marketing budget of the year, then you can present them your tool. Rick Tousseyn:  yeah the search query is what is influencer marketing Dmitriy Grankin:  Okay, so if you have some somewhat like 10 minutes, we'll just... Dmitriy Grankin:  Yeah, 10 minutes is okay. Dmitriy Grankin:  Yeah, we'll just run a quick test and that will be it. Dmitriy Grankin:  but we first we need to yeah the thing is that okay let's let's keep this thing okay this yeah I have an explanation of what is my product so it could create an article that's not about some arbitrary idea from the internet but about my specific thing but we can we can skip this for now We will just create a generic article for... Dmitriy Grankin:  Can you see my screen now? Dmitriy Grankin:  Yeah, I can see your screen. Dmitriy Grankin:  Okay. Dmitriy Grankin:  This command will scrape search results and all the pages. Dmitriy Grankin:  We will get rid of this thing now and let's have a look what will be the result. Dmitriy Grankin:  Here are the results. Dmitriy Grankin:  Post-influence marketing strategy guide for 2024. Dmitriy Grankin:  You know all you need to know to plan, execute. Dmitriy Grankin:  So these are the top 10 queries right now, right? Dmitriy Grankin:  yes top 10 results and their uh and the page content okay very cool okay so here's the result and it is some service service part of it which is a kind of thinking process behind and here is the article itself and yeah this is the the smallest the simplest model so we've got more powerful than that but yeah let me just just it to you okay yeah and it's got all the the title this description URL I'll text for image this thing is also created. Can I try? Rick Tousseyn:  yeah i will output really quickly yeah and try to um maybe output Three versions or two, one that is like, um, around 1000 words. Rick Tousseyn:  And one that is around 3000 words, because like the, I don't know if you remember seeing it entire screen because the surfer recommendation here I believe it was the original was around 3,000 words. Rick Tousseyn:  But we adjusted it because, look, so the one on the first position Sprout Social has around 4,000 words, 2,500. Rick Tousseyn:  So let's say 3,000 words is the average to rank really well. Rick Tousseyn:  Well, but I don't think that sometimes it's very strange because, look, this one is ranking with 1,000 words. Rick Tousseyn:  JOHN MUELLER, OK. Rick Tousseyn:  So it'

In [ ]:
points_by_meeting

{}